In [ ]:
import csv
import pandas as pd

Part 1: Data collection/curation + parsing

We have decided to use several sets of data: _ and a collection of homicide rates across the world.
The number of guns possessed by civilians in each country cannot stand alone as an indicator of gun violence, as people
kill other people--guns do not. Therefore, we also found a collection of homicide rates in 2016 around the world, courtesy
of the United Nations (https://dataunodc.un.org/data/homicide/Homicide%20rate%20by%20mechanisms).

NOTE: should we look for suicide by gun rates, too? What about population?

In [ ]:
with open('united_nations_homicide_rate_by_mechanisms_2017.csv', newline='') as csvfile:
    r = csv.reader(csvfile, delimiter=',')
    for row in r:
        print(', '.join(row))

Part 2: Data management/representation

In [ ]:
#stuff

Part 3: Exploratory data analysis

In [ ]:
#stuff

Part 4: Hypothesis testing & machine learning

In [ ]:
#wat

Part 5: Communication of insights attained

In [ ]:
#um